In [7]:
import sqlite3
import numpy as np
import string
import random
import os

In [8]:
def make_connection():
    db_name = "epidemic.db"
    try:
        con = sqlite3.connect(db_name)
        con.row_factory = sqlite3.Row
        cursor = con.cursor()
    except Exception as e:
        print(e)
        print(f"Error connecting to the database")
        return False, False
    return con, cursor

In [9]:
def randomNumber(be, en):
    return np.random.randint(be, en)

In [10]:
def randomString(size, chars = string.ascii_lowercase):
    return ''.join(random.choice(chars) for x in range(size))

In [18]:
def updateStats(values, con, cursor, stats):
    """
    diseaseid, ncases, ndeaths, spreadprob 
    """
    print('heyyy')
    diseaseid = values[1]
    spreadprob = random.random()
    new_values = [diseaseid, 1, values[2], spreadprob]
    
    query = f"""
        SELECT * from Stats
        WHERE diseaseid = {diseaseid};
            """
    try:
        cursor.execute(query)
        print('first query')
    except Exception as e:
        return_val = str(e) + "Error in executing query for updating stats"
    data = cursor.fetchone()
    print('frist query done')
    print(data)
    if not data:
        query = """
                INSERT INTO Stats
                VALUES (?, ?, ?, ?);
                """
        stats[diseaseid] = new_values[1:]
        print('new query\n\n')
    else:
        query = f"""
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + {values[2]}
                WHERE diseaseid = {diseaseid};
                """
        print('Old update\n')
        new_values = False
        stats.update({diseaseid: [stats[diseaseid][0]+1, stats[diseaseid][1] + values[2], spreadprob]})
    print(f"Doing query {query} and values {new_values}")
    try:
        if new_values:
            cursor.execute(query, new_values)
        else:
            cursor.execute(query)
        con.commit()
    except Exception as e:
        return_val = str(e) + "Can't execute the query."
    return return_val
        

In [26]:
def populateDB(values, type = None, stats = None):
    con, cursor = make_connection()
    return_val = "Done"
    
    if type == 'Hospital':
        query = f"""
                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        """
    elif type == 'Diseases':
        query = f"""
                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        """
    elif type == 'Drugs':
        query = f"""
                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?, ?);
        """
    elif type == 'Cases':
        query = f"""
                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        """
        try:
            cursor.execute(query, values)
            con.commit()
            updateStats(values, con, cursor, stats)
        except:
            print("comeon")
            con.close()
    try:
        cursor.execute(query, values)
        con.commit()
    except Exception as e:
        return_val = str(e)
    finally:
        con.close()
    return return_val

# Populating Hospital

In [13]:
be = 10000
en = 99999

hosp_id = []
hosp_userid = []
hosp_name = []
hosp_loc = []

for i in range(500):
    ID = randomNumber(be, en)
    Userid = randomNumber(be, en)
    Name = randomString(randomNumber(6, 12))
    Location = randomString(randomNumber(10, 25))
    
    hosp_id.append(ID)
    hosp_loc.append(Location)
    
    a = populateDB(tuple([ID, Name, Location, Userid]), 'Hospital')

# Populating Diseases

In [14]:
disease_id = []
disease_name = []
disease_mos = []
disease_sym = []

for i in range(50):
    id = randomNumber(be, en)
    name = randomString(randomNumber(6, 10))
    mos = random.choice(['air', 'water', 'food', 'animal'])
    sym = randomString(100)
    
    disease_id.append(id)
    disease_name.append(name)
    disease_mos.append(mos)
    disease_sym.append(sym)
    
    a = (populateDB(tuple([id, name, mos, sym]), 'Diseases'))

# Populating Drugs

In [15]:
drug_id = []
drug_name = []
drug_req = []
drug_avail =[]
disease_drug = {}

for i in range(len(disease_id)):
    curDisease = disease_id[i]
    for j in range(randomNumber(1,4)):
        id = randomNumber(be, en)
        name = randomString(randomNumber(6, 12))
        req = randomNumber(6, 15)
        avail = randomNumber(1500, 3000)
        
        drug_id.append(id)
        drug_name.append(name)
        drug_req.append(req)
        drug_avail.append(avail)
        
        #storing drug ids corresponding to each disease
        if(curDisease in disease_drug):
            disease_drug[curDisease].append(id)
        else:
            disease_drug[curDisease] = [id]
            
        a = populateDB(tuple([id, name, req, avail, disease_id[i]]), 'Drugs')

In [25]:
!cp backup.db epidemic.db

In [17]:
!ls

 backup.db     flask_app.py	    sqlite-epidemic.ipynb
 epidemic.db  'Populate DB.ipynb'   state_cluster_list.txt


# Populating Cases

In [27]:
# id- auto-incremental
case_date = []
case_diseaseid = []
case_death = []
case_location = []

#stats
stats = {}
#for each disease id in stats, we will store ncases, ndeaths and spreadprob

for i in range(len(disease_id)):
    curDisease = disease_id[i]
    for j in range(randomNumber(20, 40)):
        date = str(randomNumber(1, 30)) + '-' + str(randomNumber(1, 12)) + '-' + str(randomNumber(2018, 2020))
        death = randomNumber(0, 2)
        location = random.choice(hosp_loc)
        
        case_date.append(date)
        case_diseaseid.append(curDisease)
        case_death.append(death)
        case_location.append(location)
        a = populateDB(tuple([date, curDisease, death, location]), 'Cases', stats)

heyyy
first query
frist query done
None
new query


Doing query 
                INSERT INTO Stats
                VALUES (?, ?, ?, ?);
                 and values [90204, 1, 1, 0.5202356843392139]
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 90204;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 90204;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 90204;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
    

heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 95688;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 95688;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 95688;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 95688;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
    

comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 92573;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 92573;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 92573;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 92573;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing quer

comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 74103;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 74103;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 74103;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 74103;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing quer

heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 19086;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 19086;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 19086;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 19086;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
    

...

heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 70547;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 70547;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 70547;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 70547;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
    

comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 44395;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 44395;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 44395;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 44395;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing quer

heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 44945;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 44945;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 44945;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 44945;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
    

heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 78313;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 78313;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 78313;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 78313;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
    

heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 36052;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 1
                WHERE diseaseid = 36052;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 36052;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + 0
                WHERE diseaseid = 36052;
                 and values False
comeon
heyyy
first query
frist query done
Old update

Doing query 
    

In [35]:
# LOCATION is an array of 500 random locations coordinates in INDIA
# HOSPITAL_ID is an array of 500 random IDs, each corresponding to a location
# DISEASE_ID is an array of 50 diseases

In [ ]:
# 1. Hospital
